# Вебинар 4. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install implicit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 48.0 MB/s eta 0:00:00


---

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

/usr/local/lib/python3.10/dist-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: CUDA driver version is insufficient for CUDA runtime version (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Rec_systems/retail_train.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [ ]:
item_features = pd.read_csv('/content/drive/MyDrive/Rec_systems/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [ ]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [ ]:
n_items_before = data_train['item_id'].nunique()

data_train = prefilter_items(data_train, 5000, item_features)

n_items_after = data_train['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

<ipython-input-8-b90550c0de0b>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))


Decreased # items from 86865 to 5001


In [ ]:
user_item_matrix = pd.pivot_table(data_train,
                                  index='user_id', columns='item_id',
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count',
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

user_item_matrix.head(3)

item_id,117847,818981,819255,819308,819400,819487,819590,819594,819840,819845,...,15926775,15926844,15926886,15972074,15972298,15972565,15972790,16100266,16729299,16729415
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [ ]:
user_item_matrix = bm25_weight(user_item_matrix.T).T  # Применяется к item-user матрице !

In [ ]:
%%time

model = AlternatingLeastSquares(factors=20,
                                regularization=0.001,
                                iterations=15,
                                calculate_training_loss=True,
                                num_threads=4)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

  0%|          | 0/15 [00:00<?, ?it/s]

CPU times: user 2min 7s, sys: 3min 48s, total: 5min 55s
Wall time: 3min 38s


-----

# Домашнее задание

### 1. Перенесите метрики в модуль src.metrics.py

In [ ]:
import pandas as pd
import numpy as np


# функции являющиеся метриками:


#hit_rate: вычисляет показатель попадания.
#Возвращает 1, если хотя бы один элемент из рекомендованного списка присутствует в списке покупок, и 0 в противном случае.
def hit_rate(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return (flags.sum() > 0) * 1


#hit_rate_at_k: вычисляет показатель попадания на первых k позициях.
#Использует функцию hit_rate для оценки только первых k элементов из рекомендованного списка.
def hit_rate_at_k(recommended_list, bought_list, k=5):
    return hit_rate(recommended_list[:k], bought_list)


#precision: вычисляет точность.
#Определяет долю элементов из рекомендованного списка, которые присутствуют в списке покупок.
def precision(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)


#precision_at_k: вычисляет точность на первых k позициях.
#Использует функцию precision для оценки только первых k элементов из рекомендованного списка.
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)


#money_precision_at_k: вычисляет точность с учетом стоимости элементов.
#Умножает бинарные флаги на цены рекомендованных элементов и делит сумму на общую стоимость рекомендаций.
def money_precision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_recommended.sum()


#recall: вычисляет полноту. Определяет долю элементов из списка покупок, которые присутствуют в рекомендованном списке.
def recall(recommended_list, bought_list):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(bought_list)


#recall_at_k: вычисляет полноту на первых k позициях. Использует функцию recall для оценки только первых k элементов из рекомендованного списка.
def recall_at_k(recommended_list, bought_list, k=5):
    return recall(recommended_list[:k], bought_list)


#money_recall_at_k: вычисляет полноту с учетом стоимости элементов.
#Умножает бинарные флаги на цены рекомендованных элементов и делит сумму на общую стоимость покупок.
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
    flags = np.isin(recommended_list, bought_list)
    return np.dot(flags, prices_recommended).sum() / prices_bought.sum()


#ap_k: вычисляет среднюю точность на первых k позициях.
#Вычисляет точность на каждой позиции из рекомендованного списка до k и возвращает среднее значение.
def ap_k(recommended_list, bought_list, k=5):
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    recommended_list = recommended_list[recommended_list <= k]

    relevant_indexes = np.nonzero(np.isin(recommended_list, bought_list))[0]
    if len(relevant_indexes) == 0:
        return 0
    amount_relevant = len(relevant_indexes)

    sum_ = sum(
        [precision_at_k(recommended_list, bought_list, k=index_relevant + 1) for index_relevant in relevant_indexes])
    return sum_ / amount_relevant

In [ ]:
# создаём папку src в текущем рабочем каталоге и переносим туда функции,
# которые уже определены в текущем Jupyter Notebook,
# и выполним перенос функций в файл src/metrics.py.


import os
import inspect
import shutil

# Создание папки src
src_dir = '/content/drive/MyDrive/Rec_systems/src'
os.makedirs(src_dir, exist_ok=True)

# Список функций для переноса
functions_to_move = [
    hit_rate,
    hit_rate_at_k,
    precision,
    precision_at_k,
    money_precision_at_k,
    recall,
    recall_at_k,
    money_recall_at_k,
    ap_k
]

# Перенос функций в src.metrics.py
with open(os.path.join(src_dir, 'metrics.py'), 'a', encoding='utf-8') as f:
    for function in functions_to_move:
        function_source = inspect.getsource(function)
        f.write(function_source)
        f.write('\n')


----------------------------------------------------------------------------------------------------------------------------------------------

### 2. Перенесите функцию prefilter_items в модуль src.utils.py

In [ ]:
'''
Функция prefilter_items выполняет предварительную фильтрацию данных с целью подготовки данных для рекомендаций.
Она выполняет следующие действия:

- Удаляет неинтересные для рекомендаций категории товаров на основе информации о категориях (item_features).
- Редкие категории, в которых количество уникальных товаров меньше 150, считаются неинтересными, и все товары из этих категорий удаляются из данных.
- Удаляет слишком дешевые товары, так как на них не заработаем.
-Товары, у которых стоимость (sales_value) деленная на количество продаж (quantity) меньше или равна 2, удаляются из данных.
- Удаляет слишком дорогие товары. Товары, у которых стоимость (sales_value) больше 50, удаляются из данных.

Оставляет только топ-N популярных товаров на основе суммарного количества продаж (quantity).
Топ-N товаров с наибольшим количеством продаж остаются в данных, остальные удаляются. Значение N задается параметром take_n_popular.
Заменяет item_id для всех товаров, которые не входят в топ-N популярных товаров, на фиктивный item_id 999999.
Это позволяет учесть факт покупки любого товара, который не попал в топ-N, как покупку фиктивного товара.
'''
import pandas as pd
import numpy as np

def prefilter_items(data, take_n_popular=5000, item_features=None):
    # Уберем не интересные для рекоммендаций категории (department):
    if item_features is not None:
        department_size = pd.DataFrame(item_features. \
                                       groupby('department')['item_id'].nunique(). \
                                       sort_values(ascending=False)).reset_index()

        department_size.columns = ['department', 'n_items']
        rare_departments = department_size[department_size['n_items'] < 150].department.tolist()
        items_in_rare_departments = item_features[
            item_features['department'].isin(rare_departments)].item_id.unique().tolist()

        data = data[~data['item_id'].isin(items_in_rare_departments)]

    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб:
    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))
    data = data[data['price'] > 2]

    # Уберем слишком дорогие товары:
    data = data[data['price'] < 50]

    # Возьмём топ по популярности:
    popularity = data.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

    top = popularity.sort_values('n_sold', ascending=False).head(take_n_popular).item_id.tolist()


    # Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
    data.loc[~data['item_id'].isin(top), 'item_id'] = 999999

    return data

def postfilter_items(user_id, recommendations):
    pass


In [ ]:
# Перенесём функцию prefilter_items в модуль src.utils.py

functions_to_move = [
    prefilter_items
]

# Перенос функций в src.utils.py
with open(os.path.join(src_dir, 'utils.py'), 'a', encoding='utf-8') as f:
    for function in functions_to_move:
        function_source = inspect.getsource(function)
        f.write(function_source)
        f.write('\n')

----

### 3. Создайте модуль src.recommenders.py. Напишите код для класса ниже
(задание обсуждали на вебинаре, для первой функции практически сделали) и положите его в src.recommenders.py


In [ ]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать

        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data: pd.DataFrame):
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)
        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model

    @staticmethod
    def fit_own_recommender(user_item_matrix):
      """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
      own_recommender = ItemItemRecommender(K=1, num_threads=4)
      own_recommender.fit(user_item_matrix)

      return own_recommender

    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        # Получаем идентификатор пользователя из его имени
        user_id = self.userid_to_id[user]

        # Получаем список похожих пользователей
        similar_users = self.own_recommender.similar_users(user_id, N+1)

        # Удаляем исходного пользователя из списка похожих пользователей
        similar_users = similar_users[1:]

        # Получаем список товаров, купленных похожими пользователями
        items = []
        for similar_user_id in similar_users:
            recs = self.own_recommender.recommend(similar_user_id, self.user_item_matrix.T.tocsr(), N=1)
            items.append(self.id_to_itemid[recs[0][0]])

        # Удаляем дубликаты и возвращаем список рекомендаций
        res = list(set(items))

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res


    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # Получаем идентификатор пользователя из его имени
        user_id = self.userid_to_id[user]

        # Получаем топ-N товаров, купленных пользователем
        top_items = self.user_item_matrix.loc[user_id].sort_values(ascending=False).head(N)

        # Получаем похожие товары для каждого из топ-N товаров
        similar_items = []
        for item_id, score in top_items.iteritems():
            recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)
            similar_items.extend([self.id_to_itemid[rec[0]] for rec in recs[1:]])

        # Удаляем дубликаты и возвращаем список рекомендаций
        res = list(set(similar_items))

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res


In [ ]:
# Перенесём class MainRecommender в модуль src.recommenders.py
import os

functions_to_move = [
    MainRecommender
]

# Перенос классов/функций в src/recommenders.py в двоичном формате
with open(os.path.join(src_dir, 'recommenders.py'), 'ab') as f:
    for function in functions_to_move:
        function_source = inspect.getsource(function)
        f.write(function_source.encode('utf-8'))
        f.write(b'\n')


OSError: ignored

In [ ]:
import os

file_path = src_dir +"/recommenders.py"  # Path to the file to be deleted

# Check if the file exists
if os.path.exists(file_path):
    os.remove(file_path)
    print("File deleted successfully.")
else:
    print("File not found.")


File deleted successfully.


#### возникла ошибка OSError: source code not available.
#### В некоторых случаях функция inspect.getsource() может не смочь получить исходный код класса из-за ограничений или специфики окружения выполнения.

#### В этом случае проще просто вручную скопировать и вставить код класса MainRecommender в файл src/recommenders.py.
#### Либо просто создть строку с исходным кодом класса MainRecommender и записывает его в файл src/recommenders.py:

In [ ]:
import os

# MainRecommender class as a string
MainRecommenderCode = '''
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    """Рекоммендации, которые можно получить из ALS

    Input
    -----
    user_item_matrix: pd.DataFrame
        Матрица взаимодействий user-item
    """

    def __init__(self, data, weighting=True):

        # your_code. Это не обязательная часть. Но если вам удобно что-либо посчитать тут - можно это сделать

        self.user_item_matrix = self.prepare_matrix(data)  # pd.DataFrame
        self.id_to_itemid, self.id_to_userid, \
            self.itemid_to_id, self.userid_to_id = self.prepare_dicts(self.user_item_matrix)

        if weighting:
            self.user_item_matrix = bm25_weight(self.user_item_matrix.T).T

        self.model = self.fit(self.user_item_matrix)
        self.own_recommender = self.fit_own_recommender(self.user_item_matrix)

    @staticmethod
    def prepare_matrix(data: pd.DataFrame):
        user_item_matrix = pd.pivot_table(data,
                                          index='user_id', columns='item_id',
                                          values='quantity',
                                          aggfunc='count',
                                          fill_value=0
                                          )

        user_item_matrix = user_item_matrix.astype(float)
        return user_item_matrix

    @staticmethod
    def prepare_dicts(user_item_matrix):
        """Подготавливает вспомогательные словари"""

        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        id_to_itemid = dict(zip(matrix_itemids, itemids))
        id_to_userid = dict(zip(matrix_userids, userids))

        itemid_to_id = dict(zip(itemids, matrix_itemids))
        userid_to_id = dict(zip(userids, matrix_userids))

        return id_to_itemid, id_to_userid, itemid_to_id, userid_to_id

    @staticmethod
    def fit(user_item_matrix, n_factors=20, regularization=0.001, iterations=15, num_threads=4):
        """Обучает ALS"""

        model = AlternatingLeastSquares(factors=n_factors,
                                        regularization=regularization,
                                        iterations=iterations,
                                        num_threads=num_threads)
        model.fit(csr_matrix(user_item_matrix).T.tocsr())

        return model


    def get_similar_users_recommendation(self, user, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""

        # Получаем идентификатор пользователя из его имени
        user_id = self.userid_to_id[user]

        # Получаем список похожих пользователей
        similar_users = self.own_recommender.similar_users(user_id, N+1)

        # Удаляем исходного пользователя из списка похожих пользователей
        similar_users = similar_users[1:]

        # Получаем список товаров, купленных похожими пользователями
        items = []
        for similar_user_id in similar_users:
            recs = self.own_recommender.recommend(similar_user_id, self.user_item_matrix.T.tocsr(), N=1)
            items.append(self.id_to_itemid[recs[0][0]])

        # Удаляем дубликаты и возвращаем список рекомендаций
        res = list(set(items))

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res


    def get_similar_items_recommendation(self, user, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""

        # Получаем идентификатор пользователя из его имени
        user_id = self.userid_to_id[user]

        # Получаем топ-N товаров, купленных пользователем
        top_items = self.user_item_matrix.loc[user_id].sort_values(ascending=False).head(N)

        # Получаем похожие товары для каждого из топ-N товаров
        similar_items = []
        for item_id, score in top_items.iteritems():
            recs = self.model.similar_items(self.itemid_to_id[item_id], N=2)
            similar_items.extend([self.id_to_itemid[rec[0]] for rec in recs[1:]])

        # Удаляем дубликаты и возвращаем список рекомендаций
        res = list(set(similar_items))

        assert len(res) == N, 'Количество рекомендаций != {}'.format(N)
        return res

'''

# Create the recommenders.py file in Python file format
with open(os.path.join(src_dir, 'recommenders.py'), 'w', encoding='utf-8') as f:
    f.write("# -*- coding: utf-8 -*-\n")
    f.write(MainRecommenderCode)

### 4. Проверьте, что все модули корректно импортируются

Проверка, что все работает

In [ ]:
sys.path.append('/content/drive/MyDrive/Rec_systems/src')

In [ ]:
from metrics import precision_at_k, recall_at_k
from utils import prefilter_items
from recommenders import MainRecommender


In [ ]:
from recommenders import *